# Jane Street Market Prediction
[Kaggle Jane Street Market Prediction](https://www.kaggle.com/c/jane-street-market-prediction/overview)


In [1]:
import pandas as pd
import numpy as np
import datatable as dt
X = dt.fread('jane-street-market-prediction/train.csv').to_pandas()

In [2]:
X = X[X.date > 85]
y = np.where((X.resp > 0), 1, 0)
drop_cols = ['date', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp', 'ts_id']
X.drop(drop_cols, axis=1, inplace=True)
X['cross_41_42_43'] = X['feature_41'] + X['feature_42'] + X['feature_43']
X['cross_1_2'] = X['feature_1'] / (X['feature_2'] + 1e-5)

In [3]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X, y)

In [4]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
}

In [5]:
gbm = lgb.train(params, lgb_train)

[LightGBM] [Info] Number of positive: 935677, number of negative: 926920
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.062609 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 33663
[LightGBM] [Info] Number of data points in the train set: 1862597, number of used features: 133
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502351 -> initscore=0.009403
[LightGBM] [Info] Start training from score 0.009403


In [6]:
gbm.save_model('lightgbmmodel.txt')

y_pred = gbm.predict(Xt, num_iteration=gbm.best_iteration)